# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Wed Mar 11 07:16:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 440.48.02    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   42C    P8     1W / 215W |   1788MiB /  7974MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.test.is_gpu_available(cuda_only=True))
device_lib.list_local_devices()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8034786465358909470,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13772661904993777233
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5480775680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8336380964433791501
 physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4817022749254415174
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.1091, 0.0178, 0.2500],
        [0.1409, 0.9612, 0.0325],
        [0.8944, 0.3869, 0.9657],
        [0.8131, 0.5454, 0.2587],
        [0.6570, 0.0147, 0.1361]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

248 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

78.2 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.0962, 0.3125, 0.7327, 0.5982, 0.4624],
        [0.4655, 0.4890, 0.9603, 0.4339, 0.0524],
        [0.9294, 0.9639, 0.6312, 0.1752, 0.7721],
        [0.5533, 0.3656, 0.9329, 0.8796, 0.9513],
        [0.4949, 0.0972, 0.2892, 0.7570, 0.2847]], device='cuda:0')
tensor([[0.0962, 0.3125, 0.7327, 0.5982, 0.4624],
        [0.4655, 0.4890, 0.9603, 0.4339, 0.0524],
        [0.9294, 0.9639, 0.6312, 0.1752, 0.7721],
        [0.5533, 0.3656, 0.9329, 0.8796, 0.9513],
        [0.4949, 0.0972, 0.2892, 0.7570, 0.2847]], dtype=torch.float64)


In [10]:
%%timeit
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

11.4 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.4303, 0.7364, 0.1235, 0.7786, 0.7036],
        [0.3256, 0.4515, 0.7994, 0.9814, 0.7705],
        [0.2292, 0.5194, 0.4354, 0.3964, 0.5804],
        [0.8855, 0.5156, 0.9321, 0.9555, 0.4150],
        [0.0640, 0.0665, 0.1170, 0.9547, 0.2668]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[1.0966e-03, 3.5866e-04, 4.0044e-04, 3.2466e-04, 2.3044e-04],
        [3.5866e-04, 9.7424e-04, 2.8649e-04, 8.2904e-04, 2.0482e-04],
        [4.0044e-04, 2.8649e-04, 5.4179e-04, 1.2729e-04, 9.4659e-05],
        [3.2466e-04, 8.2904e-04, 1.2729e-04, 1.3005e-03, 6.6951e-06],
        [2.3044e-04, 2.0482e-04, 9.4659e-05, 6.6950e-06, 1.3420e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[1.0966e-03, 3.5866e-04, 4.0044e-04, 3.2466e-04, 2.3044e-04],
        [3.5866e-04, 9.7424e-04, 2.8649e-04, 8.2904e-04, 2.0482e-04],
        [4.0044e-04, 2.8649e-04, 5.4179e-04, 1.2729e-04, 9.4659e-05],
        [3.2466e-04, 8.2904e-04, 1.2729e-04, 1.3005e-03, 6.6951e-06],
        [2.3044e-04, 2.0482e-04, 9.4659e-05, 6.6950e-06, 1.3420e-03]],
       dtype=torch.float64)
